# バイナリ法の実装

**結局どっちが左右かわからないので、指数の上位ビットから考えるか下位ビットから考えるかで命名した。**

## 目的
べき乗を高速に計算する。  

---

## 指数の上位ビットから考えていく方式
### アルゴリズム
指数を2進として扱う。  
ビット列長を取得して、最上位ビットのみ1のビットマスクを作成する。  
ビットマスクを1ずつずらしながら以下の処理を行う。  
- 現在までの合計を2乗する。  
- もしビットマスクと指数のAND演算結果が0でない場合、現在までの値に底を乗算する。  

以上の手順を繰り返すことにより実現をする。

In [1]:
# 単純なベキ乗
def upper_binary(base, exponent):
    
    # 特殊な場合はあらかじめ省いておこう。
    if not isinstance(exponent, int):
        raise Exception("exponent must be int.")
    
    if exponent == 0:
        return 1
    elif exponent < 0:
        raise Exception("upper_binary() does'nt work support when exponent smaller than zero.")
    
    # 最上位ビットが1のマスクを作成
    mask = 1 << exponent.bit_length() - 1
    
    ans = 1
    
    # ビットマスクが移動し終わるまで、繰り返す。
    while mask:
        ans *= ans
        
        if exponent & mask:
            ans *= base
        
        # ビットマスクを移動
        mask >>= 1
        
    return ans

---

### ベキ乗の剰余計算の場合
乗算の度に剰余演算を繰り返すことにより計算を行う。

---

In [5]:
# ベキ乗の剰余計算
def upper_binary_mod(base, exponent, mod):
    # 特殊な場合はあらかじめ省いておこう。
    if not isinstance(exponent, int):
        raise Exception("exponent must be int.")
    
    if exponent == 0:
        return 1
    elif exponent < 0:
        raise Exception("upper_binary() does'nt work support when exponent smaller than zero.")
        
    if not isinstance(mod, int):
        raise Exception("mod must be int.")
        
    if mod <= 0:
        raise Exception("mod must be larger than zero.")
        
    # 最上位ビットが1のマスクを作成
    mask = 1 << exponent.bit_length() - 1
    
    ans = 1
    
    # ビットマスクが移動し終わるまで、繰り返す。
    while mask:
        ans = (ans * ans) % mod
        
        if exponent & mask:
            ans = (ans * base) % mod
        
        # ビットマスクを移動
        mask >>= 1
        
    return ans

## 指数の下位ビットから考えていく方式
### アルゴリズム
指数を2進表記して、ビット列長を求める。  
ビット列長の分だけマスクを左にシフトさせながら底を2乗していき、マスクと指数のAND演算が0でないときにその時の底のベキ乗の値を結果に乗算する。  

---

In [13]:
# 単純なベキ乗を求める。
def lower_binary(base, exponent):
    if not isinstance(exponent, int):
        raise Exception("exponent must be int")
    
    if exponent == 1:
        return 1
    
    if exponent < 1:
        raise Exception("exponent must be larger than zero")
        
    max_mask = 1 << (exponent.bit_length() - 1)
    
    mask = 1
    
    ans = 1
    current_base = base
    
    while max_mask >= mask:
        if mask & exponent != 0:
            ans *= current_base
        
        current_base *= current_base
        
        mask <<= 1
    
    return ans

In [16]:
# ベキ乗の剰余計算
def lower_binary_mod(base, exponent, mod):
    if not isinstance(exponent, int):
        raise Exception("exponent must be int")
    
    if exponent == 1:
        return 1
    
    if exponent < 1:
        raise Exception("exponent must be larger than zero.")
        
    if not isinstance(mod, int):
        raise Exception("mod must be int.")
        
    if mod < 1:
        raise Exception("mod must be larger than zero.")
        
    max_mask = 1 << (exponent.bit_length() - 1)
    
    mask = 1
    
    ans = 1
    current_base = base
    
    while max_mask >= mask:
        if mask & exponent != 0:
            ans = (ans * current_base) % mod 
        
        current_base *= current_base
        
        mask <<= 1
    
    return ans

## 以下作成した関数のベンチマークのためのテスト
### 方法
ランダムな系列を作成。  
組み込みの関数と作成した2種の関数でそれぞれ比較を行う。  

In [81]:
def test_benchmark():
    from random import randint
    from time import time
    arg_list = [{'base': randint(1, 100), 'exponent': randint(1, 100)} for i in range(10000)]
    
    upper_time = 0
    for d in arg_list:
        start = time()
        upper_binary(d['base'], d['exponent'])
        upper_time += time() - start
    print('upper_binary(): ' + str(upper_time * 1000) + 'ms')
    
    lower_time = 0
    for d in arg_list:
        start = time()
        lower_binary(d['base'], d['exponent'])
        lower_time += time() - start
    print('lower_binary() : ' + str(lower_time * 1000) + 'ms')
    
    builtin_time = 0
    for d in arg_list:
        start = time()
        d['base'] ** d['exponent']
        builtin_time += time() - start
    print('builtin                : ' + str(builtin_time * 1000) + 'ms')

In [91]:
if __name__ == '__main__':
#     print(upper_binary(3, 5))
#     print(upper_binary_mod(3, 5, 4))
#     print(lower_binary(3, 5))
#     print(lower_binary_mod(3, 5, 4))
    test_benchmark()

upper_binary(): 122.76482582092285ms
lower_binary() : 115.73076248168945ms
builtin                : 25.571823120117188ms


In [26]:
3 ** 5

243